<a href="https://colab.research.google.com/github/massivetexts/saddl-dataset/blob/main/WorkingWithDatasetInSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Run this cell to download SaDDL Dataset, install dependencies
#@markdown and create a DuckDB database. This is just *one way* to work with the
#@markdown data - the dataset parquet files can be used by various tools.
## Download Dataset
!mkdir dataset
!gsutil cp -r gs://saddl-data/dataset/0-0-1-prerelease/* ./dataset/
!sed -i s/\\/tmp\\/saddl-db-export/dataset/g dataset/load.sql #rewrite absolute path

# Install duckdb
!pip install duckdb

# connect to duckdb with write access and import data
import duckdb
with duckdb.connect('saddl.duckdb', read_only=False) as con:
    con.execute("IMPORT DATABASE 'dataset'")

## Simple Dataset Access

In [34]:
con = duckdb.connect('saddl.duckdb', read_only=True)
con.execute("CREATE INDEX htid_idx ON meta (htid);")
con.execute("CREATE INDEX target_clean_idx ON clean_predictions (target);")
con.execute("CREATE INDEX candidate_clean_idx ON clean_predictions (candidate);")

In [26]:
con.execute("SHOW TABLES").fetch_df()

,name
0,clean_predictions
1,clusters
2,htid_idx
3,manifestation_predictions
4,manifestation_stats
5,meta
6,target_clean_idx
7,work_predictions
8,work_stats


In [27]:
con.execute("SELECT * FROM clean_predictions LIMIT 1").fetch_df()

,target,candidate,swsm,swde,wp_dv,partof,contains,OVERLAPS,author,simdiff,grsim,randdiff,relatedness,count
0,aeu.ark:/13960/t0000bj8z,hvd.32044055347264,0.001621,0.00003,0.003333,0.000129,0.000106,0.000062,0.782082,0.138136,0.002091,0.072409,0.050686,2


In [59]:
con.execute('''SELECT meta.title, meta.author, clusters.*, work_stats.* 
FROM clusters
JOIN work_stats ON clusters.work_id == work_stats.work_id
JOIN meta ON meta.htid == clusters.htid
WHERE label_count > 3 AND work_stats.gov_prop < .4 AND work_stats.serial_prop < .4
LIMIT 1
''').fetch_df()

,title,author,htid,work_id,man_id,label_count,gov_count,serial_count,gov_prop,serial_prop,include,best_centroid,best_centroid_pd,best_median,best_median_pd
0,The life of Lord Nelson by Robert Southey.,"Southey, Robert, 1774-1843.",aeu.ark:/13960/t0000qm8d,21284,715227,7,0,0,0.0,0.0,True,mdp.39015063625381,mdp.39015063625381,mdp.39015063625381,mdp.39015063625381
